In [1]:
import time

import tensorflow as tf
import numpy as np
import vizdoom as vd

from skimage.transform import rescale
from tqdm import trange
from IPython.display import HTML


In [2]:
#Specify the game scenario and the screen format/resolution

game = vd.DoomGame()
game.set_screen_format(vd.ScreenFormat.BGR24)
game.set_screen_resolution(vd.ScreenResolution.RES_320X240)
game.set_depth_buffer_enabled(False)
game.load_config('take_cover.cfg')

down_sample_ratio = 0.25
width = int(game.get_screen_width()*down_sample_ratio)
height = int(game.get_screen_height()*down_sample_ratio)
channels = game.get_screen_channels() + int(game.is_depth_buffer_enabled())

#Specify the available actions in the scenario

available_actions = game.get_available_buttons()
actions = [list(ohe) for ohe in list(np.identity(len(available_actions)))]
num_actions = len(available_actions)

#Specify the Q-network learning parameters

frame_delay = 12
buffer_size = 50000
epochs = 400
steps_per_epoch = 2000
learning_rate = 0.0025
gamma = 0
start_epsilon = 1.0
end_epsilon = 0.1
batch_size = 100
load_model = False
save_model = True
model_dir = './checkpoints/take_cover.ckpt'
num_ckpts = 40


In [3]:
#Create a buffer object that holds a set of training experiences (state-action-reward tuples)

class Buffer():
    def __init__(self, size=1000):
        self.buffer = list()
        self.length = len(self.buffer)
        self.size = size
        
#Add a new experience to the buffer (remove the oldest experience if the buffer is already full)
        
    def add_experience(self, experience):
        if self.length + 1 >= self.size:
            self.buffer[0:(self.length + 1) - self.size] = []
        
        self.buffer.append(experience)
        self.length = len(self.buffer)
            
#Return a batch of experience arrays randomly sampled from the buffer
            
    def sample_buffer(self, sample_size):
        sample = np.random.randint(self.length, size=sample_size)
        s1 = np.concatenate([self.buffer[idx][0] for idx in sample], axis=0)
        a = np.array([self.buffer[idx][1] for idx in sample])
        r = np.array([self.buffer[idx][2] for idx in sample])
        s2 = np.concatenate([self.buffer[idx][3] for idx in sample], axis=0)
        terminal = np.array([self.buffer[idx][4] for idx in sample], dtype=np.int32)
        
        return s1, a, r, s2, terminal

#Downsample and normalize an image array representing the game state at a given time stamp

def preprocess(image, down_sample_ratio=1):
    if float(down_sample_ratio) != 1.0:
        image = rescale(image=image,
                        scale=(down_sample_ratio,
                               down_sample_ratio),
                        mode='reflect')
    image = image.astype(np.float32)
    image = np.expand_dims(image, axis=0)

    return image

#Test the agent using a currently training or previously trained model

def test_agent(model, num_episodes, load_model, depth, training=True, session=None, model_dir=None):
    if load_model == True:
        sess = tf.Session()
        print('Loading model from', model_dir)
        tf.train.Saver().restore(sess, model_dir)
        
#Require an existing session if a pretrained model isn't provided
        
    elif load_model == False:
        sess = session

    game.set_sound_enabled(True)
    episode_rewards = list()
    
#Avoid reinitializing the game if this was already done by the training process
    
    if training == False:
        game.init()

    for i in range(num_episodes):
        game.new_episode()
    
        while not game.is_episode_finished():
            state = game.get_state()
            
            if depth == False:
                state_buffer = np.moveaxis(state.screen_buffer, 0, 2)
            
            elif depth == True:
                depth_buffer = state.depth_buffer
                state_buffer = np.stack((state.screen_buffer,
                                         depth_buffer), axis=-1)
                
            state1 = preprocess(state_buffer, down_sample_ratio)
            action = model.choose_action(sess, state1)[0]
            reward = game.make_action(actions[action])
            
#Add a delay between each time step so that the episodes occur at normal speed

            time.sleep(0.02)
        
        episode_rewards.append(game.get_total_reward())
        print('Test Episode {} Reward: {}'.format(i + 1, game.get_total_reward()))
        time.sleep(1)
    
#Avoid ending the game so that the training process can continue
    
    if training == False:
        game.close()
    
    return np.mean(episode_rewards)


In [4]:
#Create a Q-network to estimate values and choose actions for a given state

class Q_network():
    def __init__(self, network_name, height, width, channels, learning_rate=0.001):
        self.learning_rate = learning_rate
        self.s_t = tf.placeholder(tf.float32,
                                  shape=[None, height, width, channels],
                                  name=network_name + '_state'
                                 )
        self.a_t = tf.placeholder(tf.int32,
                                  shape=[None],
                                  name=network_name + '_action'
                                 )
        self.Q_target = tf.placeholder(tf.float32,
                                       shape=[None, num_actions],
                                       name=network_name + '_Q_target'
                                      )

        self.input_layer = tf.reshape(self.s_t,
                                      [-1, height, width, channels],
                                      name=network_name + '_input_layer'
                                     )
        self.conv1 = tf.layers.conv2d(inputs=self.input_layer,
                                      filters=32,
                                      kernel_size=[8, 8],
                                      strides=[4, 4],
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      name=network_name + '_conv1_layer'
                                     )
        self.conv2 = tf.layers.conv2d(inputs=self.conv1,
                                      filters=64,
                                      kernel_size=[4, 4],
                                      strides=[2, 2],
                                      padding='valid',
                                      activation=tf.nn.relu,
                                      name=network_name + '_conv2_layer'
                                     )
        self.flatten = tf.layers.flatten(self.conv2,
                                         name=network_name + '_flatten'
                                        )
        self.dense = tf.layers.dense(inputs=self.flatten,
                                      units=512,
                                      activation=tf.nn.relu,
                                      name=network_name + '_dense1_layer'
                                    )
        self.Q_values = tf.layers.dense(inputs=self.dense,
                                        units=len(actions),
                                        activation=None,
                                        name=network_name + '_output_layer'
                                       )        
    
        self.best_action = tf.argmax(self.Q_values, 1)
        self.loss = tf.losses.mean_squared_error(self.Q_values,
                                                 self.Q_target)
        self.adam = tf.train.AdamOptimizer(learning_rate=self.learning_rate,
                                           name=network_name + '_adam'
                                          )
        self.train = self.adam.minimize(self.loss)
        
    def update_lr(self):
        self.learning_rate = 0.98*self.learning_rate
        
        return self.learning_rate

    def calculate_loss(self, session, s, q):
        L, _ = session.run([self.loss, self.train],
                           feed_dict={self.s_t: s,
                                      self.Q_target: q})
    
        return L

#Return the array of Q-values and the best action associated with a given state

    def get_Q_values(self, session, s):
        Q = session.run(self.Q_values,
                        feed_dict={self.s_t: s})

        return Q
    
    def choose_action(self, session, s):
        a = session.run(self.best_action,
                        feed_dict={self.s_t: s})
    
        return a
    
#Create a list of variable update operations

def update_graph(variables):
    update_ops = list()
    
#Assign weight values from the network created first to the one created second
    
    for idx, variable in enumerate(variables[:len(variables)//2]):
        op = variable.assign(variables[idx + len(variables)//2].value())
        update_ops.append(op)
    
    return update_ops

#Update the target network parameters to match those of the online network

def update_target(ops, session):
    for op in update_ops:
        session.run(op)


In [5]:
#For each time step, collect the following data:
#The current game state
#The action that was taken taken
#The reward obtained from the chosen action
#The next game state (store the first game state if the previous action ends the episode)
#A variable indicating whether the episode is over yet


tf.reset_default_graph()

#Instantiate the target network before the online network so that it's updated correctly

target_net = Q_network(network_name='target',
                       learning_rate=learning_rate,
                       height=height,
                       width=width,
                       channels=channels)
DQN = Q_network(network_name='online',
                learning_rate=learning_rate,
                height=height,
                width=width,
                channels=channels)

exp_buffer = Buffer(size=buffer_size)
session = tf.Session()
saver = tf.train.Saver(max_to_keep=num_ckpts, reshape=True)
weights = tf.trainable_variables()

update_ops = update_graph(weights)

if load_model == True:
    print('Loading model from', model_dir)
    tf.train.Saver().restore(session, model_dir)
    
elif load_model == False:
    session.run(tf.global_variables_initializer())

game.set_sound_enabled(False)
game.init()

t = 0
epoch_rank = list()
epoch_rank_depth = list()


In [6]:
#Accumulate experiences in the buffer using an epsilon-greedy strategy with three training phases

for epoch in range(epochs):
    epoch_rewards = list()
    
    for step in trange(steps_per_epoch, leave=True):
        experience = list()
        game.new_episode()
        
        while not game.is_episode_finished():
            state = game.get_state()
            
#Substitute an array of zeros for the depth buffer if that setting is disabled

            if game.is_depth_buffer_enabled() == False:
                state1_buffer = np.moveaxis(state.screen_buffer, 0, 2)
            else:
                depth_buffer = state.depth_buffer
                state1_buffer = np.stack((state.screen_buffer,
                                          depth_buffer), axis=-1)
                
            state1 = preprocess(state1_buffer, down_sample_ratio)
            
#Explore the environment by choosing random actions with 100% probability for the first phase of training

            if epoch < 0.2*epochs:
                action = np.random.randint(num_actions)
            
#Increase the probability of greedily choosing an action by a constant amount at each epoch in the second phase
            
            elif epoch < 0.9*epochs:
                epsilon = start_epsilon - (epoch + 1 - 0.2*epochs)*(start_epsilon-end_epsilon)/(0.7*epochs)
            
                if np.random.uniform(0, 1) <= epsilon:
                    action = np.random.randint(num_actions)
                else:
                    action = DQN.choose_action(session, state1)[0]

#Select a random action with 10% probability in the final phase of training
                
            else:
                if np.random.uniform(0, 1) <= end_epsilon:
                    action = np.random.randint(num_actions)
                else:
                    action = DQN.choose_action(session, state1)[0]

            reward = game.make_action(actions[action], frame_delay)
            done = game.is_episode_finished()
            
            if done == False:
                state = game.get_state()
                
                if game.is_depth_buffer_enabled() == False:
                    state2_buffer = np.moveaxis(state.screen_buffer, 0, 2)
                else:
                    depth_buffer = state.depth_buffer
                    state2_buffer = np.stack((state.screen_buffer,
                                              depth_buffer), axis=-1)
                
                state2 = preprocess(state2_buffer, down_sample_ratio)
                
            elif done == True:
                state2 = state1
        
#Add the experience obtained from each time step to the buffer

            t += 1
            exp_buffer.add_experience((state1, action, reward, state2, done))
        
#Sample a minibatch from the buffer if there are enough experiences in the buffer

        if exp_buffer.length > batch_size:
            s1, a, r, s2, terminal = exp_buffer.sample_buffer(batch_size)
            
#Get the target values from the target Q-network
            
            target_Q = np.max(target_net.get_Q_values(session, s2), axis=1)
            
#Train the online Q-network by using a minibatch to update the action-value function
            
            Q2 = DQN.get_Q_values(session, s1)
            Q2[np.arange(batch_size), a] = r + gamma*(1 - terminal)*target_Q
            DQN.calculate_loss(session, s1, Q2)
            
        epoch_rewards.append(game.get_total_reward())
        
#Increase the discount factor at each epoch until it reaches 0.99
    
    if gamma < 0.99:
        gamma = 1-.98*(1-gamma)
    elif gamma >= 0.99:
        gamma = 0.99
        
#Decrease the learning rate at each epoch

    DQN.update_lr()
    target_net.update_lr()
    
    print('Epoch {} Mean Reward: {}'.format(epoch + 1, np.mean(epoch_rewards)))
    
#Update the target network every 10 epochs
    
    if (epoch + 1) % 10 == 0 and epoch > 0:
        update_target(update_ops, session)
        
#Save the model and test the agent for 10 episodes every 20 epochs
    
    if (epoch + 1) % 10 == 0 and epoch > 0:
        if save_model == True:
            checkpoint = model_dir + '-' + str(epoch + 1)
            print('Epoch {} Model saved to {}'.format(epoch + 1, model_dir))
            saver.save(session, model_dir, global_step=epoch + 1)
            
        update_target(update_ops, session)

#Test the agent both with and without the depth buffer given
        
        print('Epoch {} test:'.format(epoch + 1))
        test_reward = test_agent(DQN, num_episodes=10,
                                 training=True,
                                 load_model=False,
                                 depth=False,
                                 session=session,
                                 model_dir=model_dir)
        print('Average Test Reward:', test_reward)
        
        epoch_rank_depth.append((test_reward, epoch + 1))
        epoch_rank.append((test_reward, epoch + 1))
        
#Return a sorted list of epoch checkpoints based on average test episode reward
        
print(sorted(epoch_rank, reverse=True))
print('{} time steps experienced during training'.format(t))
game.close()
    

100%|██████████| 2000/2000 [16:17<00:00,  2.05it/s]


Epoch 1 Mean Reward: 389.122


100%|██████████| 2000/2000 [21:24<00:00,  1.56it/s]


Epoch 2 Mean Reward: 395.356


100%|██████████| 2000/2000 [22:12<00:00,  1.50it/s]


...

Epoch 398 Mean Reward: 583.7705


100%|██████████| 2000/2000 [22:12<00:00,  1.50it/s]


Epoch 399 Mean Reward: 615.035


100%|██████████| 2000/2000 [22:06<00:00,  1.51it/s]


Epoch 400 Mean Reward: 618.7065
Epoch 400 Model saved to ./checkpoints/take_cover.ckpt
Epoch 400 test:
Test Episode 1 Reward: 438.0
Test Episode 2 Reward: 274.0
Test Episode 3 Reward: 196.0
Test Episode 4 Reward: 421.0
Test Episode 5 Reward: 421.0
Test Episode 6 Reward: 421.0
Test Episode 7 Reward: 421.0
Test Episode 8 Reward: 585.0
Test Episode 9 Reward: 421.0
Test Episode 10 Reward: 421.0
Average Test Reward: 401.9
[(934.0, 90), (608.3, 240), (544.8, 150), (456.7, 310), (451.7, 180), (446.6, 290), (446.5, 40), (429.4, 210), (422.8, 260), (420.4, 350), (401.9, 400), (394.4, 120), (387.2, 130), (378.4, 300), (378.2, 110), (354.3, 280), (350.7, 330), (348.7, 370), (339.2, 250), (337.1, 230), (336.5, 270), (332.4, 160), (325.3, 360), (315.8, 10), (315.3, 390), (312.3, 380), (302.2, 170), (298.8, 320), (270.7, 340), (265.6, 80), (263.1, 140), (236.5, 100), (231.4, 30), (230.6, 220), (229.5, 70), (224.8, 200), (221.3, 60), (208.9, 50), (207.1, 190), (193.4, 20)]
31843289 time steps experie

In [9]:
"""
Delete the "checkpoint" file in model directory and manually specify a checkpoint file path
if a screen resolution that's different from the one that the agent was trained on is to be used.
"""

#Get a list of checkpoints saved during training

ckpts = tf.train.get_checkpoint_state('checkpoints').all_model_checkpoint_paths

#Test the trained model from a certain checkpoint by only choosing actions with a greedy strategy

for checkpoint in ckpts:
    test_reward = test_agent(DQN, num_episodes=20,
                                 training=False,
                                 load_model=True,
                                 depth=False,
                                 model_dir=checkpoint)
    print('Epoch {} Average Test Reward: {}'.format(checkpoint, test_reward))


Loading model from checkpoints\take_cover.ckpt-10
INFO:tensorflow:Restoring parameters from checkpoints\take_cover.ckpt-10
Test Episode 1 Reward: 284.0
Test Episode 2 Reward: 299.0
Test Episode 3 Reward: 793.0
Test Episode 4 Reward: 299.0
Test Episode 5 Reward: 299.0
Test Episode 6 Reward: 584.0
Test Episode 7 Reward: 570.0
Test Episode 8 Reward: 299.0
Test Episode 9 Reward: 299.0
Test Episode 10 Reward: 299.0
Test Episode 11 Reward: 299.0
Test Episode 12 Reward: 258.0
Test Episode 13 Reward: 830.0
Test Episode 14 Reward: 299.0
Test Episode 15 Reward: 220.0
Test Episode 16 Reward: 244.0
Test Episode 17 Reward: 319.0
Test Episode 18 Reward: 282.0
Test Episode 19 Reward: 299.0
Test Episode 20 Reward: 289.0
Epoch checkpoints\take_cover.ckpt-10 Average Test Reward: 368.2
Loading model from checkpoints\take_cover.ckpt-20
INFO:tensorflow:Restoring parameters from checkpoints\take_cover.ckpt-20
Test Episode 1 Reward: 219.0
Test Episode 2 Reward: 219.0
Test Episode 3 Reward: 219.0
Test Episode

Test Episode 10 Reward: 476.0
Test Episode 11 Reward: 108.0
Test Episode 12 Reward: 263.0
Test Episode 13 Reward: 476.0
Test Episode 14 Reward: 476.0
Test Episode 15 Reward: 181.0
Test Episode 16 Reward: 517.0
Test Episode 17 Reward: 476.0
Test Episode 18 Reward: 476.0
Test Episode 19 Reward: 126.0
Test Episode 20 Reward: 643.0
Epoch checkpoints\take_cover.ckpt-110 Average Test Reward: 417.65
Loading model from checkpoints\take_cover.ckpt-120
INFO:tensorflow:Restoring parameters from checkpoints\take_cover.ckpt-120
Test Episode 1 Reward: 255.0
Test Episode 2 Reward: 493.0
Test Episode 3 Reward: 357.0
Test Episode 4 Reward: 493.0
Test Episode 5 Reward: 493.0
Test Episode 6 Reward: 493.0
Test Episode 7 Reward: 388.0
Test Episode 8 Reward: 192.0
Test Episode 9 Reward: 493.0
Test Episode 10 Reward: 493.0
Test Episode 11 Reward: 129.0
Test Episode 12 Reward: 233.0
Test Episode 13 Reward: 493.0
Test Episode 14 Reward: 662.0
Test Episode 15 Reward: 356.0
Test Episode 16 Reward: 256.0
Test Epi

Epoch checkpoints\take_cover.ckpt-210 Average Test Reward: 383.1
Loading model from checkpoints\take_cover.ckpt-220
INFO:tensorflow:Restoring parameters from checkpoints\take_cover.ckpt-220
Test Episode 1 Reward: 232.0
Test Episode 2 Reward: 117.0
Test Episode 3 Reward: 232.0
Test Episode 4 Reward: 232.0
Test Episode 5 Reward: 232.0
Test Episode 6 Reward: 124.0
Test Episode 7 Reward: 224.0
Test Episode 8 Reward: 232.0
Test Episode 9 Reward: 232.0
Test Episode 10 Reward: 277.0
Test Episode 11 Reward: 232.0
Test Episode 12 Reward: 232.0
Test Episode 13 Reward: 135.0
Test Episode 14 Reward: 248.0
Test Episode 15 Reward: 232.0
Test Episode 16 Reward: 181.0
Test Episode 17 Reward: 232.0
Test Episode 18 Reward: 232.0
Test Episode 19 Reward: 232.0
Test Episode 20 Reward: 208.0
Epoch checkpoints\take_cover.ckpt-220 Average Test Reward: 214.9
Loading model from checkpoints\take_cover.ckpt-230
INFO:tensorflow:Restoring parameters from checkpoints\take_cover.ckpt-230
Test Episode 1 Reward: 668.0


Test Episode 7 Reward: 267.0
Test Episode 8 Reward: 324.0
Test Episode 9 Reward: 341.0
Test Episode 10 Reward: 132.0
Test Episode 11 Reward: 324.0
Test Episode 12 Reward: 324.0
Test Episode 13 Reward: 324.0
Test Episode 14 Reward: 225.0
Test Episode 15 Reward: 324.0
Test Episode 16 Reward: 126.0
Test Episode 17 Reward: 324.0
Test Episode 18 Reward: 162.0
Test Episode 19 Reward: 193.0
Test Episode 20 Reward: 324.0
Epoch checkpoints\take_cover.ckpt-320 Average Test Reward: 239.45
Loading model from checkpoints\take_cover.ckpt-330
INFO:tensorflow:Restoring parameters from checkpoints\take_cover.ckpt-330
Test Episode 1 Reward: 288.0
Test Episode 2 Reward: 535.0
Test Episode 3 Reward: 535.0
Test Episode 4 Reward: 344.0
Test Episode 5 Reward: 126.0
Test Episode 6 Reward: 535.0
Test Episode 7 Reward: 611.0
Test Episode 8 Reward: 535.0
Test Episode 9 Reward: 119.0
Test Episode 10 Reward: 307.0
Test Episode 11 Reward: 321.0
Test Episode 12 Reward: 535.0
Test Episode 13 Reward: 535.0
Test Episod